# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [47]:
#store file path into variable
file= "../WeatherPy/output_data/cities.csv"

#read file
read_file= pd.read_csv(file)

#create dataframe and removed unneccesary columns
cities_df = pd.DataFrame(read_file)
cities_df= cities_df.drop(['Unnamed: 0'], axis=1)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,narsaq,60.92,-46.05,47.53,68,100,2.48,GL,1598568909
1,kavieng,-2.57,150.80,83.16,72,49,5.26,PG,1598568910
2,hobart,-42.88,147.33,52.00,57,75,18.34,AU,1598568676
3,nanortalik,60.14,-45.24,49.46,71,100,12.75,GL,1598568912
4,kapaa,22.08,-159.32,86.00,58,20,12.75,US,1598568856


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [33]:
#configure gmaps and store locations and humidity 
gmaps.configure(api_key=g_key)
locations = cities_df[["Lat", "Lng"]]
humidity= cities_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [49]:
#create the ideal weather dataframe
ideal_weather= cities_df.loc[(cities_df["Max Temp"]<80) & (cities_df["Max Temp"]>70) & 
                            (cities_df["Wind Speed"]<10) & (cities_df["Cloudiness"]==0)]
ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,roebourne,-20.78,117.13,75.63,36,0,5.28,AU,1598568938
116,caravelas,-17.71,-39.25,71.67,85,0,7.83,BR,1598569222
127,cadiz,36.50,-5.75,75.99,63,0,3.00,ES,1598569235
185,fengkou,30.08,113.27,73.47,85,0,4.65,CN,1598569300
224,lithakia,37.72,20.83,73.40,73,0,2.98,GR,1598569343
230,misratah,32.38,15.09,77.54,72,0,3.83,LY,1598569350
250,suluq,31.67,20.25,72.55,85,0,8.16,LY,1598569372
306,sao joao da barra,-21.64,-41.05,71.33,88,0,8.23,BR,1598569299
342,lianzhou,24.78,112.38,72.18,74,0,1.45,CN,1598569473
363,renqiu,38.70,116.09,71.19,82,0,2.04,CN,1598569497


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [110]:
#Store ideal weather into hotel_df variable
hotel_df=ideal_weather

#create a new column 
hotel_df["Hotel Name"]=""

#set parameters to search for hotels 
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius":"5000",  
    "key": g_key,
    "keyword":"Hotel"
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get location from df
    lat = row['Lat']
    lng = row['Lng']
    location =f"{lat},{lng}"
    # update location key value
    params['location'] = f"{lat},{lng}"

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:         
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print(f"Missing field/result for {location}... skipping.")
    

C:\Users\TEMIDAYOAKINSANYA\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\TEMIDAYOAKINSANYA\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result for 30.08,113.27... skipping.
Missing field/result for 31.67,20.25... skipping.


In [112]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [114]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))